# Imports/Setup

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from pathlib import Path
import spotipy
from spotipy.oauth2 import SpotifyOAuth


%matplotlib inline

dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)

conn = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(os.getenv('DB_USER'),
                                                               os.getenv('DB_PASS'), 
                                                               os.getenv('DB_IP'), 
                                                               os.getenv('DB_PORT'), 
                                                               os.getenv('DB_NAME')))

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REDIRECT_URI = 'https://google.com'
SCOPE = "playlist-modify-private"
PLAYLIST_ID = '0gXnKF7IfnEpeuqYAgUimy'

token = SpotifyOAuth(scope=SCOPE,
                    client_id=CLIENT_ID,
                    client_secret=CLIENT_SECRET,
                    show_dialog=True,
                    redirect_uri='https://google.com')
sp = spotipy.Spotify(auth_manager=token)

# Download/Setup Data

In [8]:
dreampop_data = pd.read_sql("SELECT * FROM dreampop;", conn)
dreampop_data.head()

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,name,artist,genre
0,0,0.482,0.866,7,-4.534,1,0.0293,0.000368,0.11600,0.247,...,audio_features,5gEni4dmBqme8fyPcyWYiK,spotify:track:5gEni4dmBqme8fyPcyWYiK,https://api.spotify.com/v1/tracks/5gEni4dmBqme...,https://api.spotify.com/v1/audio-analysis/5gEn...,187364,4,Kate's Not Here - Day Wave & Lawrence Rothman ...,Lawrence Rothman,dreampop
1,1,0.543,0.498,9,-10.152,1,0.0678,0.692000,0.43200,0.078,...,audio_features,0l4cR2AZ18MktMd0BU3zwh,spotify:track:0l4cR2AZ18MktMd0BU3zwh,https://api.spotify.com/v1/tracks/0l4cR2AZ18Mk...,https://api.spotify.com/v1/audio-analysis/0l4c...,241097,4,Tenderness,Jay Som,dreampop
2,2,0.644,0.756,7,-8.342,1,0.0273,0.567000,0.15800,0.151,...,audio_features,04DdNbTmZAgHfNP6FdY3Sa,spotify:track:04DdNbTmZAgHfNP6FdY3Sa,https://api.spotify.com/v1/tracks/04DdNbTmZAgH...,https://api.spotify.com/v1/audio-analysis/04Dd...,264955,4,Plum,Widowspeak,dreampop
3,3,0.508,0.798,1,-5.412,0,0.0259,0.256000,0.20200,0.701,...,audio_features,22gjW0ccBALzqASrryR7Dw,spotify:track:22gjW0ccBALzqASrryR7Dw,https://api.spotify.com/v1/tracks/22gjW0ccBALz...,https://api.spotify.com/v1/audio-analysis/22gj...,213481,4,Opportunity,Kevin Krauter,dreampop
4,4,0.562,0.724,2,-6.787,1,0.0622,0.203000,0.00115,0.129,...,audio_features,4kDqNIQiBwkqDs6jWWcHAM,spotify:track:4kDqNIQiBwkqDs6jWWcHAM,https://api.spotify.com/v1/tracks/4kDqNIQiBwkq...,https://api.spotify.com/v1/audio-analysis/4kDq...,166453,4,What Does It Mean To Me?,Oberhofer,dreampop


# Create Playlist

In [12]:
track_ids = list(set(list(dreampop_data['id'])))

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

track_chunks = list(chunks(track_ids, 100))

94

In [13]:
for chunk in track_chunks:
    sp.playlist_add_items(PLAYLIST_ID, chunk)